In [1]:
import numpy as np
from rascal.representations import SphericalExpansion as SPH
import ase.io
import tqdm
from nice.blocks import *
from nice.utilities import *
from matplotlib import pyplot as plt
from sklearn.linear_model import BayesianRidge
import torch
torch.set_num_threads(1)
import torch.nn
from code_pytorch import *
import time
import numpy as np
from sympy.physics.wigner import wigner_3j
from sympy.physics.quantum.spin import Rotation as rot
from rascal.representations import SphericalInvariants as SOAP
from torch2trt import torch2trt

In [2]:
clebsch = nice.clebsch_gordan.ClebschGordan(4)

In [3]:
now = clebsch.precomputed_[3, 2, 1, :3 * 2 + 1, :2 * 2 + 1]
now = torch.tensor(now, dtype = torch.float)
print(now.shape)
model = ClebschCombiningSingle(now, 1)
#model = model.cuda()

torch.Size([7, 5])


In [4]:
first = torch.tensor(np.random.rand(200, 170, 7), dtype = torch.float)
second = torch.tensor(np.random.rand(200, 170, 5), dtype = torch.float)

#first = first.cuda()
#second = second.cuda()

In [5]:
begin = time.time()
with torch.no_grad():
    result = model(first, second)
end = time.time()
print(end - begin)

1.6737635135650635


In [6]:
print(result.shape)

torch.Size([200, 28900, 3])


In [7]:
first = torch.tensor(np.random.rand(200, 170 * 170, 7), dtype = torch.float)
second = torch.tensor(np.random.rand(200, 170 * 170, 5), dtype = torch.float)
model = ClebschCombiningSingleUnrolled(now, 1)

In [8]:
begin = time.time()
with torch.no_grad():
    result = model(first, second)
end = time.time()
print(end - begin)

1.545365333557129


In [9]:
index = []
mask = []
for m1 in range(now.shape[0]):
    for m2 in range(now.shape[1]):
        if (m1+ m2 < (2 * 1 + 1)):
            index.append(m1 + m2)
            mask.append(True)
        else:
            mask.append(False)
            
mask = torch.tensor(mask, dtype = torch.bool)
index = torch.LongTensor(index)

In [10]:
begin = time.time()
with torch.no_grad():
    X1 = first[:, :, :, None]
    X2 = second[:, :, None, :]
    #print(self.l1, self.l2, X1.shape, X2.shape)
    mult = X1 * X2
    mult = mult * now

    mult = mult.reshape(mult.shape[0], mult.shape[1], -1)
  
    result = torch.zeros([mult.shape[0], mult.shape[1], 2 * 1 + 1])

    result = result.index_add_(2, index, mult[:, :, mask])       
    end = time.time()
print(end - begin)

1.5336074829101562


In [11]:
begin = time.time()
with torch.no_grad():
    X1 = first[:, :, :, None]
    X2 = second[:, :, None, :]
    #print(self.l1, self.l2, X1.shape, X2.shape)
    mult = X1 * X2
    mult = mult * now
    
    #mult = mult.reshape(mult.shape[0], mult.shape[1], -1)
  
    result = torch.zeros([mult.shape[0], mult.shape[1], 2 * 1 + 1])
   
   
    for m1 in range(now.shape[0]):
        for m2 in range(now.shape[1]):
            if (m1 + m2 < 2 * 1 + 1):
                result[:, :, m1 + m2] += mult[:, :, m1, m2]
    print(result.shape)
    #result = result.index_add_(2, index, mult[:, :, mask])       
    end = time.time()
print(end - begin)

torch.Size([200, 28900, 3])
1.4115169048309326


In [12]:
print(model)

ClebschCombiningSingleUnrolled()


In [13]:
model_j = torch.jit.trace(model, [first, second])

In [14]:
begin = time.time()
with torch.no_grad():
    result = model_j(first, second)
end = time.time()
print(end - begin)

1.5585017204284668


In [15]:
model_j = torch.jit.script(model, [first, second])

/home/pozdn/.local/lib/python3.6/site-packages/torch/jit/_script.py:891: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  "`optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead"


In [16]:
begin = time.time()
with torch.no_grad():
    result = model_j(first, second)
end = time.time()
print(end - begin)

1.5498745441436768


In [17]:
first = first.cuda()
second = second.cuda()
model = model.cuda()

In [22]:
begin = time.time()
for _ in range(10):
    with torch.no_grad():
        result = model(first, second)
        total = torch.sum(result).cpu().data.numpy()
end = time.time()
print(end - begin)

0.5191397666931152


In [19]:
index = index.cuda()
mask = mask.cuda()
now = now.cuda()

In [20]:
print(first.shape)
print(second.shape)

torch.Size([200, 28900, 7])
torch.Size([200, 28900, 5])


In [24]:
first_t = torch.FloatTensor(np.random.rand(7, 200, 28900)).cuda()
second_t = torch.FloatTensor(np.random.rand(5, 200, 28900)).cuda()

In [25]:
print(first_t.shape)
print(second_t.shape)

torch.Size([7, 200, 28900])
torch.Size([5, 200, 28900])


In [26]:
print(now.shape)

torch.Size([7, 5])


In [29]:
#m first loops only required
begin = time.time()
for _ in range(10):
    with torch.no_grad():
        #X1 = first_t[:, None, :, :]
        #X2 = second_t[None, :, :, :]
        #print(X1.shape)
        #print(X2.shape)
        #print(self.l1, self.l2, X1.shape, X2.shape)
        #mult = X1 * X2
        #print(mult.shape)
        #mult = mult * now[:, :, None, None]
        #print(mult.shape)                

        #mult = mult.reshape(-1, mult.shape[2], mult.shape[3])
        #print(mult.shape)
        result = torch.zeros([ 2 * 1 + 1, first_t.shape[1], first_t.shape[2]], device = 'cuda')
        for m1 in range(now.shape[0]):
            for m2 in range(now.shape[1]):
                if (m1 + m2 < 2 * 1 + 1):
                    #result[m1 + m2, :, :] += mult[m1, m2, :, :]
                    result[m1 + m2, :, :] += first_t[m1] * second_t[m2] * now[m1, m2]
        total = torch.sum(result)
        total = total.cpu().data.numpy()
        #print(now[1, 3])
        #print(torch.sum(first_t[1] * second_t[3] * now[1, 3]))
        #print(mult.shape)
        #result = result.cpu().data.numpy()

        #result = result.index_add_(0, index, mult[mask])       
end = time.time()
print(end - begin)

0.10175013542175293


In [88]:
#m first loops
begin = time.time()
for _ in range(10):
    with torch.no_grad():
        X1 = first_t[:, None, :, :]
        X2 = second_t[None, :, :, :]
        #print(X1.shape)
        #print(X2.shape)
        #print(self.l1, self.l2, X1.shape, X2.shape)
        mult = X1 * X2
        #print(mult.shape)
        mult = mult * now[:, :, None, None]
        #print(mult.shape)                

        #mult = mult.reshape(-1, mult.shape[2], mult.shape[3])
        #print(mult.shape)
        result = torch.zeros([ 2 * 1 + 1, mult.shape[2], mult.shape[3]], device = 'cuda')
        for m1 in range(now.shape[0]):
            for m2 in range(now.shape[1]):
                if (m1 + m2 < 2 * 1 + 1):
                    result[m1 + m2, :, :] += mult[m1, m2, :, :]
        total = torch.sum(result)
        total = total.cpu().data.numpy()
        #print(mult.shape)
        #result = result.cpu().data.numpy()

        #result = result.index_add_(0, index, mult[mask])       
end = time.time()
print(end - begin)

0.37432265281677246


In [69]:
print(index.shape)

torch.Size([6])


In [91]:
#m first index_add
begin = time.time()
for _ in range(10):
    with torch.no_grad():
        X1 = first_t[:, None, :, :]
        X2 = second_t[None, :, :, :]
        #print(X1.shape)
        #print(X2.shape)
        #print(self.l1, self.l2, X1.shape, X2.shape)
        mult = X1 * X2
        #print(mult.shape)
        mult = mult * now[:, :, None, None]
        #print(mult.shape)                

        mult = mult.reshape(-1, mult.shape[2], mult.shape[3])
        #print(mult.shape)
        result = torch.zeros([ 2 * 1 + 1, mult.shape[1], mult.shape[2]], device = 'cuda')
       
        #print(mult.shape)
        #result = result.cpu().data.numpy()

        result = result.index_add_(0, index, mult[mask]) 
        
        total = torch.sum(result)
        total = total.cpu().data.numpy()
end = time.time()
print(end - begin)

0.40592169761657715


In [94]:
#m last loops
begin = time.time()
for _ in range(10):
    with torch.no_grad():
        X1 = first[:, :, :, None]
        X2 = second[:, :, None, :]
        #print(self.l1, self.l2, X1.shape, X2.shape)
        mult = X1 * X2
        mult = mult * now
        #print(mult.shape)
        #mult = mult.reshape(mult.shape[0], mult.shape[1], -1)

        result = torch.zeros([mult.shape[0], mult.shape[1], 2 * 1 + 1], device = 'cuda')

        #print(result.shape)
        #print(mult.shape)
        #print(now.shape)
        for m1 in range(now.shape[0]):
            for m2 in range(now.shape[1]):
                if (m1 + m2 < 2 * 1 + 1):
                    result[:, :, m1 + m2] += mult[:, :, m1, m2]
        total = torch.sum(result)
        total = total.cpu().data.numpy()
        #result = result.cpu().data.numpy()
        #print(type(result))
        #result = result.index_add_(2, index, mult[:, :, mask])       
end = time.time()
print(end - begin)

0.6034359931945801


In [96]:
#m last index_add
begin = time.time()
for _ in range(10):
    with torch.no_grad():
        X1 = first[:, :, :, None]
        X2 = second[:, :, None, :]
        #print(self.l1, self.l2, X1.shape, X2.shape)
        mult = X1 * X2
        mult = mult * now
        #print(mult.shape)
        mult = mult.reshape(mult.shape[0], mult.shape[1], -1)

        result = torch.zeros([mult.shape[0], mult.shape[1], 2 * 1 + 1], device = 'cuda')
        result = result.index_add_(2, index, mult[:, :, mask])   
        #print(result.shape)
        #print(mult.shape)
        #print(now.shape)
       
        total = torch.sum(result)
        total = total.cpu().data.numpy()
        #result = result.cpu().data.numpy()
        #print(type(result))
            
end = time.time()
print(end - begin)

0.5235612392425537


In [96]:
print(now.shape)

torch.Size([7, 5])


In [11]:
model_t = torch2trt(model, [first, second])

AttributeError: 'Tensor' object has no attribute '_trt'

In [19]:
begin = time.time()
with torch.no_grad():
    result = model_t(first, second)
end = time.time()
print(end - begin)

NameError: name 'model_t' is not defined